# Coletando dados

In [1]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO, StringIO

In [2]:
payload = {'modalidade': 'Lotofácil'}
headers = {
'Accept': '*/*',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'en,pt-BR;q=0.9,pt;q=0.8,en-GB;q=0.7,en-US;q=0.6',
#'Connection': 'keep-alive',
'Host': 'servicebus2.caixa.gov.br',
'Origin': 'https://loterias.caixa.gov.br',
'Referer': 'https://loterias.caixa.gov.br/'}

#'https://servicebus2.caixa.gov.br/portaldeloterias/api/resultados/download?modalidade=Lotof%C3%A1cil'
r = requests.get('https://servicebus2.caixa.gov.br/portaldeloterias/api/resultados/download?modalidade=Lotof%C3%A1cil', data=payload, headers=headers, verify=False)

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'servicebus2.caixa.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [3]:
pdDF = pd.read_excel(BytesIO(r.content))

# Preparando ambiente para PySpark

#### Dependências para Python 3.10

In [4]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz
!pip install -q findspark

#### Configuração das Variáveis de Ambiente

In [5]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

#### Tornar PySpark Importável

In [6]:
# tornar o pyspark "importável"
import findspark
findspark.init('spark-3.4.0-bin-hadoop3')

## Iniciando Ambiente

In [7]:
# iniciar uma sessão local e importar dados do Airbnb
from pyspark.sql import SparkSession
sc = SparkSession.builder.master('local[*]').getOrCreate()

## Carregando Pandas DF para PySpark DataFrame

In [8]:
from pyspark.sql.functions import col
from pyspark.sql.functions import cast

In [9]:
#df = df.withColumn('data', col('data').cast('string'))

In [10]:
df = sc.createDataFrame(pdDF)

In [11]:
df.show(3)

+--------+------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+---------------------+-----------+-----------------+---------------------+-----------------+---------------------+-----------------+---------------------+-----------------+---------------------+-----------------+--------------------+-----------------+-----------------+-----------------------------------------------------+--------------------+
|Concurso|Data Sorteio|Bola1|Bola2|Bola3|Bola4|Bola5|Bola6|Bola7|Bola8|Bola9|Bola10|Bola11|Bola12|Bola13|Bola14|Bola15|Ganhadores 15 acertos|Cidade / UF|Rateio 15 acertos|Ganhadores 14 acertos|Rateio 14 acertos|Ganhadores 13 acertos|Rateio 13 acertos|Ganhadores 12 acertos|Rateio 12 acertos|Ganhadores 11 acertos|Rateio 11 acertos|Acumulado 15 acertos|Arrecadacao Total|Estimativa Prêmio|Acumulado sorteio especial Lotofácil da Independência|          Observação|
+--------+------------+-----+-----+-----+-----+-----+-----+-----+-----+-

In [12]:
df.printSchema()

root
 |-- Concurso: long (nullable = true)
 |-- Data Sorteio: string (nullable = true)
 |-- Bola1: long (nullable = true)
 |-- Bola2: long (nullable = true)
 |-- Bola3: long (nullable = true)
 |-- Bola4: long (nullable = true)
 |-- Bola5: long (nullable = true)
 |-- Bola6: long (nullable = true)
 |-- Bola7: long (nullable = true)
 |-- Bola8: long (nullable = true)
 |-- Bola9: long (nullable = true)
 |-- Bola10: long (nullable = true)
 |-- Bola11: long (nullable = true)
 |-- Bola12: long (nullable = true)
 |-- Bola13: long (nullable = true)
 |-- Bola14: long (nullable = true)
 |-- Bola15: long (nullable = true)
 |-- Ganhadores 15 acertos: long (nullable = true)
 |-- Cidade / UF: string (nullable = true)
 |-- Rateio 15 acertos: string (nullable = true)
 |-- Ganhadores 14 acertos: long (nullable = true)
 |-- Rateio 14 acertos: string (nullable = true)
 |-- Ganhadores 13 acertos: long (nullable = true)
 |-- Rateio 13 acertos: string (nullable = true)
 |-- Ganhadores 12 acertos: long (nulla

In [22]:
import sys
import pickle
def estimate_dataframe_size(df, sample_fraction=0.01):
    sample_df = df.sample(fraction=sample_fraction, seed=42) #.limit(1000)
    local_data = sample_df.toPandas()
    sample_size_bytes = sys.getsizeof(pickle.dumps(local_data))
    estimated_total_size = (sample_size_bytes / sample_fraction)
    return estimated_total_size

def calculate_optimal_partitions(estimated_size_bytes, target_file_size_mb=256 #128 ou 256 MB):
    target_file_size_bytes = target_file_size_mb * 1024 * 1024
    num_partitions = int(estimated_size_bytes / target_file_size_bytes)
    return max(num_partitions, 1)

In [23]:
estimated_size = estimate_dataframe_size(df)
optimal_partitions = calculate_optimal_partitions(estimated_size)

print(f"Estimated size: {estimated_size / (1024**2):.2f} MB")
print(f"Optimal partitions: {optimal_partitions}")

Estimated size: 1.38 MB
Optimal partitions: 1


In [27]:
local = sys.getsizeof(pickle.dumps(df.toPandas()))
local/( 1024**2 )

1.1788978576660156